In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

/home/fermin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def do_datasets():
        #CARGAMOS LOS DATOS

    traindata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TRAIN.txt')#reading the data
    testdata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TEST.txt')
    #print('Number of rows and columns of the training set: ',traindata.shape)
    #print('Number of rows and columns of the test set: ',testdata.shape)
    #VARIABLES DE CADA TIPO
    list_columns_test=list(testdata.columns.values)
    #Ver el nombre de las columnas para saber cuales sumar (buscar la mayor correlación)
    list_columns=list(traindata.columns.values)
    Names_Imp_cons=list_columns[0:17]
    #Importe de consumos habituales del cliente en base a sus operaciones con tarjetas y domiciliaciones más comunes
    traindata['Imp_cons_total'] = traindata[Names_Imp_cons].sum(axis=1)
    #Importe de los saldos de los distintos productos financieros.
    traindata['Imp_cons_total_2']=traindata['Imp_cons_total']**2
    traindata['Imp_cons_total_3']=traindata['Imp_cons_total']**3
    Names_Imp_Sal=list_columns[17:38]
    traindata['Imp_sal_total'] = traindata[Names_Imp_Sal].sum(axis=1)
    #Tenencia de los distintos productos financieros. Son indices entre 0 y 2 por lo que el sumatorio también valdra
    #ya que cuanta más puntuación saque el sumatorio más productos compra
    traindata['Imp_sal_total_2']=traindata['Imp_sal_total']**2
    traindata['Imp_sal_total_3']=traindata['Imp_sal_total']**3
    Names_Ind_Prod=list_columns[38:62]
    traindata['Ind_prod_total'] = traindata[Names_Ind_Prod].sum(axis=1)
    #Número de operaciones a través de los distintos productos financieros.
    traindata['Ind_prod_total_2'] = traindata['Ind_prod_total']**2
    traindata['Ind_prod_total_3'] = traindata['Ind_prod_total']**3
    Names_Num_Oper=list_columns[62:82]
    traindata['Num_Oper_total'] = traindata[Names_Num_Oper].sum(axis=1)

    traindata['Num_Oper_total_2'] = traindata['Num_Oper_total']**2
    traindata['Num_Oper_total_3'] = traindata['Num_Oper_total']**3
    #datasetfinal
    traindata['Relacion']=((traindata.Imp_sal_total+traindata.Ind_prod_total)*traindata.Num_Oper_total)-traindata.Imp_cons_total
    #df = data[['Relacion','Imp_cons_total', 'Imp_sal_total', 'Ind_prod_total','Num_Oper_total','Socio_Demo_01','Socio_Demo_02','Socio_Demo_03','Socio_Demo_04','Socio_Demo_05','Poder_Adquisitivo']].copy()

    traindata['Relacion_2'] = traindata['Relacion']**2
    traindata['Relacion_3'] = traindata['Relacion']**3
    #hacemos lo mismo con lo de test (la lista que contiene nombres nos sirve la misma)

    testdata['Imp_cons_total'] = testdata[Names_Imp_cons].sum(axis=1)
    testdata['Imp_cons_total_2'] = testdata['Imp_cons_total']**2
    testdata['Imp_cons_total_3'] = testdata['Imp_cons_total']**3
    testdata['Imp_sal_total'] = testdata[Names_Imp_Sal].sum(axis=1)
    testdata['Imp_sal_total_2'] = testdata['Imp_sal_total']**2
    testdata['Imp_sal_total_3'] = testdata['Imp_sal_total']**3
    testdata['Ind_prod_total'] = testdata[Names_Ind_Prod].sum(axis=1)
    testdata['Ind_prod_total_2'] = testdata['Ind_prod_total']**2
    testdata['Ind_prod_total_3'] = testdata['Ind_prod_total']**3
    testdata['Num_Oper_total'] = testdata[Names_Num_Oper].sum(axis=1)
    testdata['Num_Oper_total_2'] = testdata['Num_Oper_total']**2
    testdata['Num_Oper_total_3'] = testdata['Num_Oper_total']**3
    testdata['Relacion']=((testdata.Imp_sal_total+testdata.Ind_prod_total)*testdata.Num_Oper_total)-testdata.Imp_cons_total
    testdata['Relacion_2']= testdata['Relacion']**2
    testdata['Relacion_3']= testdata['Relacion']**3
    traindata.dtypes.value_counts()

    numeric_cols=traindata.select_dtypes(include=[np.number]).columns#select only numerical
    nominal_cols=traindata.select_dtypes(exclude=[np.number]).columns#select only non numerical
    #EL ID DEL CLIENTE NO NOS SIRVE PARA NADA

    #traindata['Poder_Adquisitivo_Ranges']=pd.qcut(traindata['Poder_Adquisitivo'],4,labels=[0,1,2,3])
    traindata=traindata.drop('ID_Customer',axis=1) #remove the column from train
    test_ids = testdata['ID_Customer'] #save id column from test
    testdata=testdata.drop('ID_Customer',axis=1) #remove the column from test
    nominal_cols=nominal_cols[nominal_cols!='Id'] #remove the column name from this list as well

    #CONCATENAMOS AMBOS CONJUNTOS PARA APLICAR TRANSFORMACIONES
    #Sacamos columna para el clasificador


    data=pd.concat([traindata,testdata],axis=0,ignore_index=True) #concatenate training and test set for future transformat

    stdSc = StandardScaler()

    numeric_cols=numeric_cols[numeric_cols!='Poder_Adquisitivo'] #We don't want to scale SalePrice

    data.loc[:, numeric_cols] = stdSc.fit_transform(data.loc[:, numeric_cols])

    #SEPARAMOS DE NUEVO LOS CONJUNTOS

    traindata=data.iloc[:traindata.shape[0],:] 
    testdata=data.iloc[traindata.shape[0]:,:]
    testdata=testdata.drop('Poder_Adquisitivo',axis=1) #We drop the unknown variable in the test. It was just filled with NAs

    X_traindata=traindata.drop('Poder_Adquisitivo',axis=1)
    y_traindata=traindata['Poder_Adquisitivo']
    X_traindata_Sin_totales = X_traindata.drop(['Imp_cons_total','Imp_sal_total','Ind_prod_total','Num_Oper_total','Relacion', \
                                         'Imp_cons_total_2','Imp_sal_total_2','Ind_prod_total_2','Num_Oper_total_2','Relacion_2', \
                                         'Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)

    Test_data_Sin_totales = testdata.drop(['Imp_cons_total','Imp_sal_total','Ind_prod_total','Num_Oper_total','Relacion', \
                                         'Imp_cons_total_2','Imp_sal_total_2','Ind_prod_total_2','Num_Oper_total_2','Relacion_2', \
                                         'Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)

    X_traindata_Con_totales = X_traindata.drop(['Imp_cons_total_2','Imp_sal_total_2','Ind_prod_total_2','Num_Oper_total_2','Relacion_2', \
                                         'Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)

    Test_data_Con_totales = testdata.drop(['Imp_cons_total_2','Imp_sal_total_2','Ind_prod_total_2','Num_Oper_total_2','Relacion_2', \
                                         'Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)

    X_traindata_Con_totales_y_cuadrados = X_traindata.drop(['Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)

    Test_data_Con_totales_y_cuadrados = testdata.drop(['Imp_cons_total_3','Imp_sal_total_3','Ind_prod_total_3','Num_Oper_total_3','Relacion_3'],axis=1)
    if not os.path.exists('Total'):
        os.makedirs('Total')
    if not os.path.exists('Sin_Totales'):
        os.makedirs('Sin_Totales')
    if not os.path.exists('Con_Totales'):
        os.makedirs('Con_Totales')
    if not os.path.exists('Con_Totales_y_Cuadrados'):
        os.makedirs('Con_Totales_y_Cuadrados')

    X_traindata.to_csv('./Total/X_traindata.csv', sep=',', encoding='utf-8',index=False)
    y_traindata.to_csv('./Total/y_traindata.csv', sep=',', encoding='utf-8',index=False)
    testdata.to_csv('./Total/TEST.csv', sep=',', encoding='utf-8',index=False)

    X_traindata_Sin_totales.to_csv('./Sin_Totales/X_traindata.csv', sep=',', encoding='utf-8',index=False)
    y_traindata.to_csv('./Sin_Totales/y_traindata.csv', sep=',', encoding='utf-8',index=False)
    Test_data_Sin_totales.to_csv('./Sin_Totales/TEST.csv', sep=',', encoding='utf-8',index=False)

    X_traindata_Con_totales.to_csv('./Con_Totales/X_traindata.csv', sep=',', encoding='utf-8',index=False)
    y_traindata.to_csv('./Con_Totales/y_traindata.csv', sep=',', encoding='utf-8',index=False)
    Test_data_Con_totales.to_csv('./Con_Totales/TEST.csv', sep=',', encoding='utf-8',index=False)

    X_traindata_Con_totales_y_cuadrados.to_csv('./Con_Totales_y_Cuadrados/X_traindata.csv', sep=',', encoding='utf-8',index=False)
    y_traindata.to_csv('./Con_Totales_y_Cuadrados/y_traindata.csv', sep=',', encoding='utf-8',index=False)
    Test_data_Con_totales_y_cuadrados.to_csv('./Con_Totales_y_Cuadrados/TEST.csv', sep=',', encoding='utf-8',index=False)

In [3]:
do_datasets()